In [15]:
import json
data_path = '/home/quang_ai/AdvancedLLMReasoning/checkpoints/prm_dataset_stage1.json'

with open(data_path, 'r') as f:
    data = json.load(f)
data = data['dataset']
print(f"Số lượng câu hỏi: {len(data)}")
print(f"Số lượng steps: {sum(len(item['solution_steps']) for item in data)}")
print(f"Dữ liệu mẫu: {data[0]}")

Số lượng câu hỏi: 12515
Số lượng steps: 45821
Dữ liệu mẫu: {'question': 'Martha has 18 crayons. She lost half of them, so she bought a new set of 20 crayons. How many crayons in total does Martha have after the purchase?', 'expected_answer': '29', 'solution_steps': ["Let's solve this problem using Python code.", '<llm-code>\ncrayons_initial = 18\ncrayons_lost = crayons_initial / 2\ncrayons_bought = 20\ntotal_crayons = crayons_initial + crayons_bought - crayons_lost\ntotal_crayons\n</llm-code>', '<llm-code-output>\n29.0\n</llm-code-output>', 'Thus Martha has \\boxed{29} crayons in total.']}


In [ ]:
# =========== loop mẫu ============ 
sample = data[0]
# lấy question và golden answer
question = sample['question']
golden_answer = sample['expected_answer']
# lấy các bước giải
solution_steps = sample['solution_steps']
# =========== loop steps ============ 
# tạo context (question, step_1, step_2, ..., step_n-1)

{'question': 'Martha has 18 crayons. She lost half of them, so she bought a new set of 20 crayons. How many crayons in total does Martha have after the purchase?',
 'expected_answer': '29',
 'solution_steps': ["Let's solve this problem using Python code.",
  '<llm-code>\ncrayons_initial = 18\ncrayons_lost = crayons_initial / 2\ncrayons_bought = 20\ntotal_crayons = crayons_initial + crayons_bought - crayons_lost\ntotal_crayons\n</llm-code>',
  '<llm-code-output>\n29.0\n</llm-code-output>',
  'Thus Martha has \\boxed{29} crayons in total.']}

In [ ]:
prompt = f"""Based on the problem and the partial solution provided below, continue reasoning to solve the problem.

### Question:
{question}

### Partial Solution (Steps taken so far):
{context}

### Instruction:
Act as a mathematical "completer". Continue reasoning strictly from the last step in the "Partial Solution" above.
1. Maintain logical consistency with the provided context.
2. Use Python code blocks (```python ... ```) if calculation is needed.
3. Put the final answer in \\boxed{{}} at the end.

### Solution:
"""

In [ ]:
system_context = 
for sample in data:
    question = sample['question']
    golden_answer = sample['expected_answer']
    solution_steps = sample['solution_steps']
    
    context = """
        ### Question:\n{}\n\n### Instruction:\n
        ### Solution:\n
    """
    for i in range(len(solution_steps)):
        step = solution_steps[i]
        # Sử dụng context và step để huấn luyện mô hình
        # ...
        
        # Cập nhật context bằng cách thêm step hiện tại
        context += " " + step